In [74]:
from PIL import Image
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import sys
import torch
import numpy as np
import torchvision.transforms as T

plt.rcParams["savefig.bbox"] = 'tight'
orig_img = Image.open(Path('../input/surface-crack-detection/Negative/00026.jpg'))
torch.manual_seed(0)
data_path = '../input/surface-crack-detection/'
diz_class = {'Positive':'Crack','Negative':'No crack'}

In [58]:
def load_images_from_folder(folder,class_name):
    
    images = []
    class_path = os.path.join(folder, class_name)
    for filename in os.listdir(class_path):
        img = cv2.imread(os.path.join(class_path,filename))
        images.append(img)
        if len(images)>3:
           break
    
    plt.figure(figsize=(12,12))    
    for img,x in zip(images,range(1,7)):
        plt.subplot(1,4,x)
        plt.title(f'{diz_class[class_name]}')
        plt.axis("off")
        plt.imshow(img)

In [59]:
load_images_from_folder(data_path,"Positive")
load_images_from_folder(data_path,"Negative")

In [54]:
def plot(imgs, with_orig=True, col_title=None, **imshow_kwargs):
    if not isinstance(imgs[0], list):
        # Make a 2d grid even if there's just 1 row
        imgs = [imgs]

    num_rows = len(imgs)
    num_cols = len(imgs[0]) + with_orig
    fig, axs = plt.subplots(nrows=num_rows, ncols=num_cols, squeeze=False)
    for row_idx, row in enumerate(imgs):
        row = [orig_img] + row if with_orig else row
        for col_idx, img in enumerate(row):
            ax = axs[row_idx, col_idx]
            ax.imshow(np.asarray(img), **imshow_kwargs)
            ax.set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

    if with_orig:
        axs[0, 0].set(title='Original image')
        axs[0, 0].title.set_size(8)
    if col_title is not None:
        for col_idx in range(num_cols-1):
            axs[0, col_idx+1].set(title=col_title[col_idx])
            axs[0, col_idx+1].title.set_size(8)

    plt.tight_layout()

In [71]:
np.asarray(orig_img).shape

## 1. Simple transformations

### Resize

In [92]:
resized_imgs = [T.Resize(size=size)(orig_img) for size in [32,128]]
plot(resized_imgs,col_title=["32x32","128x128"])

### GrayScale

In [76]:
gray_img = T.Grayscale()(orig_img)
plot([gray_img], cmap='gray', col_title=["Gray"])

### Normalize

In [77]:
normalized_img = T.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))(T.ToTensor()(orig_img)) 
normalized_img = [T.ToPILImage()(normalized_img)]
plot(normalized_img, col_title=["Standard normalize"])

### Random Rotation

In [78]:
rotated_imgs = [T.RandomRotation(degrees=d)(orig_img) for d in range(50,151,50)]
plot(rotated_imgs, col_title=["Rotation 50","Rotation 100","Rotation 150"])

### Center Crop

In [79]:
center_crops = [T.CenterCrop(size=size)(orig_img) for size in (128,64, 32)]
plot(center_crops,col_title=['128x128','64x64','32x32'])

### Random Crop

In [93]:
random_crops = [T.RandomCrop(size=size)(orig_img) for size in (128,64, 32)]
plot(random_crops,col_title=['128x128','64x64','32x32'])

### Gaussian Blur

In [96]:
blurred_imgs = [T.GaussianBlur(kernel_size=(51, 91), sigma=sigma)(orig_img) for sigma in (3,7)]
plot(blurred_imgs,col_title=["sigma=3","sigma=7"])

## 2. More advanced techniques

### Gaussian Noise

In [81]:
def add_noise(inputs,noise_factor=0.3):
     noisy = inputs+torch.randn_like(inputs) * noise_factor
     noisy = torch.clip(noisy,0.,1.)
     return noisy
    
noise_imgs = [add_noise(T.ToTensor()(orig_img),noise_factor) for noise_factor in (0.3,0.6,0.9)]
noise_imgs = [T.ToPILImage()(noise_img) for noise_img in noise_imgs]
plot(noise_imgs, col_title=["noise_factor=0.3","noise_factor=0.6","noise_factor=0.9"])

### Random Blocks

In [82]:
def add_random_boxes(img,n_k,size=32):
    h,w = size,size
    img = np.asarray(img)
    img_size = img.shape[1]
    boxes = []
    for k in range(n_k):
        y,x = np.random.randint(0,img_size-w,(2,))
        img[y:y+h,x:x+w] = 0
        boxes.append((x,y,h,w))
    img = Image.fromarray(img.astype('uint8'), 'RGB')
    return img

blocks_imgs = [add_random_boxes(orig_img,n_k=i) for i in (10,20)]
plot(blocks_imgs,col_title=["10 black boxes","20 black boxes"])

### Central Block

In [97]:
def add_central_region(img,size=32):
    h,w = size,size
    img = np.asarray(img)
    img_size = img.shape[1] 
    img[int(img_size/2-h):int(img_size/2+h),int(img_size/2-w):int(img_size/2+w)] = 0
    img = Image.fromarray(img.astype('uint8'), 'RGB')
    return img
  
central_imgs = [add_central_region(orig_img,size=s) for s in (32,64)]
plot(central_imgs,col_title=["32","64"])